In [2]:
import os
import pandas as pd
import numpy as np
import glob
import datetime
from datetime import datetime
import bz2
import re
import glob
from tqdm import tqdm
import random
from datetime import timedelta

# This mutes useless warnings:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [2]:
#first preprocess the data with:
# This will take a VERY long time, I recommend running in the terminal
# !python dataloader.py

In [3]:
from data_loader import NASAAirportDataset

# this will take a while
train_ds = NASAAirportDataset('KCLT', 'data/preprocess/train')



In [4]:

# Change these times and samples
def time_sampler(n_samples, start=datetime(2022, 9, 1, 0, 0, 0), end=datetime(2023, 9, 2, 23, 0)):
    i = 0
    while i < n_samples:
        yield start + timedelta(seconds=random.randint(0, int((end - start).total_seconds())))
        i += 1
def data_itr():
    for time in time_sampler(np.inf):
        yield train_ds[time]
# time_samples = [start + timedelta(seconds=random.randint(0, int((end - start).total_seconds()))) for _ in range(n_samples)]


In [14]:
import torch

for i in time_sampler(3):
    print('Flights at time', i)
    inputs, targets = train_ds[i]
    print(inputs)
    print(targets)
    # for flight_data in train_ds[i]:
    #     train model on one row of (flight_data)



Flights at time 2022-09-22 15:40:39
tensor([[ 0.0139, -1.0000, -1.0000,  ..., -1.0000,  1.0000, -1.0000],
        [ 0.0255, -1.0000, -1.0000,  ..., -1.0000,  1.0000, -1.0000],
        [ 0.0374, -1.0000, -1.0000,  ..., -1.0000,  1.0000, -1.0000],
        ...,
        [ 0.9939, -1.0000, -1.0000,  ..., -1.0000,  1.0000, -1.0000],
        [ 0.9739, -1.0000, -1.0000,  ..., -1.0000,  1.0000, -1.0000],
        [ 0.9387, -1.0000, -1.0000,  ..., -1.0000,  1.0000, -1.0000]])
tensor([1.3889e-02, 2.5463e-02, 3.7407e-02, 3.1019e-02, 2.4259e-02, 9.2593e-05,
        1.5463e-02, 7.3148e-03, 6.0741e-02, 3.7685e-02, 4.7685e-02, 1.0491e-01,
        4.6296e-02, 8.1296e-02, 1.2676e-01, 5.3148e-02, 9.4167e-02, 1.1287e-01,
        6.7593e-02, 8.8148e-02, 1.1324e-01, 1.4148e-01, 1.8046e-01, 1.2176e-01,
        1.7269e-01, 1.7111e-01, 1.6398e-01, 8.7315e-02, 1.8824e-01, 1.9056e-01,
        2.0620e-01, 1.3028e-01, 1.4833e-01, 2.2657e-01, 1.5546e-01, 1.9778e-01,
        2.2796e-01, 2.1861e-01, 1.3935e-01, 2.4870

In [5]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# dataset = NASAAirportDataset(airport_code='KJFK', data_dir='/path/to/data', lamp_df=lamp_df, to_tensor=True)


input_dim = 1326
output_dim = 4 * 3   # 15 min intervals across 3 hours
hidden_dim = 64

class SimpleFeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleFeedForward, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.layers(x)

model = SimpleFeedForward(input_dim, hidden_dim, output_dim)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 1000
model.train()

# for epoch, time in (pbar := tqdm(enumerate(time_sampler(num_epochs)))):
for epoch in (pbar := range(num_epochs)):
    running_loss = 0.0
    epoch_len = 0

    for x_batch, y_batch in data_itr():
        epoch_len = len(x_batch)
        optimizer.zero_grad()
        outputs = model(x_batch)  # forward pass
        loss = criterion(outputs, y_batch)
        loss.backward()  # backpropagate
        optimizer.step()  # update weights

        running_loss += loss.item() * x_batch.size(0)

    epoch_loss = running_loss / epoch_len
    pbar.set_description(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

print("Training completed.")




/home/miner/repos/523Proj/venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 12])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/miner/repos/523Proj/venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([117])) that is different to the input size (torch.Size([117, 12])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (12) must match the size of tensor b (117) at non-singleton dimension 1